<a href="https://colab.research.google.com/github/Ekpenyong-Esu/Deep-RL-Exercise/blob/main/frozen_lake_real_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
#!pip install gym-notebook-wrapper

#!sudo apt-get install xvfb

In [163]:
#!pip install pygame

In [164]:
#!apt-get install x11-utils > /dev/null 2>&1 
#!pip install pyglet > /dev/null 2>&1 
#!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [165]:
%matplotlib notebook
from IPython.display import clear_output
import time  # slow the game down a little bit
import gym
import numpy as np  # used for all kinds of matrix / vector operations
import matplotlib.pyplot as plt  # for plotting
from IPython.display import clear_output

In [166]:
#import gnwrapper
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [167]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery=False)

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [168]:
# Making Environment
#env = gnwrapper.Animation(gym.make('FrozenLake-v1', is_slippery=False))

In [169]:
action_size = env.action_space.n
state_size = env.observation_space.n

state_size
action_size

4

In [170]:
q_table = np.zeros([state_size, action_size])

In [171]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [172]:
q_table.shape

(16, 4)

In [173]:
# It is common to leave Hyperparameters in ALL CAPS to easily locate them

EPOCHS=20000  # number of epochs/episodes to train for
ALPHA = 0.8 # aka the learning rate
GAMMA = 0.95 # aka the discount rate
# MAX_EPISODES = 100  # optional, also defined in env setup above

In [174]:
# Exploration vs. Exploitation parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001            # Exponential decay rate for exploration prob

## Now it is time to dive into the training / Q-Table update methodology.<br />
First we will define some functions needed for training phase

* epsilon_greedy_action_selection: Is used to implement the epsilon greedy action selection routine.
* compute_next_q_value: Computes the next Q-Values according to the formula from the lecture
* reduce_epsilon: Reduces the $\epsilon$ used for the epsilon greedy algorithm

**FUNCTION TO SELECT AN ACTION**

If we simply always select the argmax() qtable value during training, we'll most likely get stuck in an explotation loop, so we'll use a random value to randomly select an action from time to time, helping the model explore , rather than exploit.

In [175]:
def epsilon_greedy_action_selection(epsilon, q_table, discrete_state):
    '''
    Returns an action for the agent. Note how it uses a random number to decide on
    exploration versus explotation trade-off.
    '''
    random_number = np.random.random()
    
    # EXPLOITATION, USE BEST Q(s,a) Value
    if random_number > epsilon:
        # Action row for a particular state
        state_row = q_table[discrete_state,:]    # This take a particular row 
                                                #and all the element in the coloumn
        # Index of highest action for state
        # Recall action is mapped to index (e.g. 0=LEFT, 1=DOWN, etc..)
        action = np.argmax(state_row)             # This return the index of the 
                                                  # maximum element
    
    # EXPLORATION, USE A RANDOM ACTION
    else:
        # Return a random 0,1,2,3 action
        action = env.action_space.sample()
        
    return action

**FUNCTION FOR Q_VALUE COMPUTATION**

**Here we have our main Q-Learning update equation, note how it takes in the old q-value, the next optimal q value, along with our current reward, and then updates the next q value accordingly.**

In [176]:
def compute_next_q_value(old_q_value, reward, next_optimal_q_value):
    
    return old_q_value +  ALPHA * (reward + GAMMA * next_optimal_q_value - old_q_value)

**FUNCTION TO REDUCE EPSILON**

**As training continues, we need to balance explotation versus exploration, we want ot make sure our agent doesn't get trapped in a cycle going from an F square to another F Square back and forth. We also don't want our agent permanently choosing random values. We'll use the function below to try to balance this.**

In [177]:
def reduce_epsilon(epsilon,epoch):
    
    return min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*epoch)

In [ ]:


# List of rewards
rewards = []

# Play 20k games
for episode in range(EPOCHS):
    # Reset the environment
    state = env.reset()
    done = False
    total_rewards = 0
    
    while not done:
        action = epsilon_greedy_action_selection(epsilon,q_table, state)

      # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        
        # Look up current/old qtable value Q(s_t,a_t)
        old_q_value =  q_table[state,action]  

        # Get the next optimal Q-Value
        next_optimal_q_value = np.max(q_table[new_state, :])  #p.max is different 
                                  # from argmax, np.max returns the highest value 
                                  # and argmax returns the index of the highest
                                  # value

        # Compute next q value
        next_q = compute_next_q_value(old_q_value, reward, next_optimal_q_value)   

        # Update Q Table
        q_table[state,action] = next_q

        
        
        total_rewards = total_rewards + reward
        
        # Our new state is state
        state = new_state


        # print(f"Epsilon is currently {epsilon}")
        # print(f"Currently on Epoch {episode}, total reward earned so far is {total_rewards}")
        # print(f"Current state {state} and performing action {action}")
        # #env.render()
        # print(q_table)
        # #time.sleep(0.01)
        # clear_output(wait=True)

        
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = reduce_epsilon(epsilon,episode) 
    rewards.append(total_rewards)
    


env.close()


**Now our agent can learn from our Q table**

In [ ]:
plt.plot(range(EPOCHS),np.cumsum(rewards))

In [ ]:

state = env.reset()
rewards = 0
for _ in range(100):
#  env.render()
    
  action = np.argmax(q_table[state])  # and chose action from the Q-Table
  state, reward, done, info = env.step(action) # Finally perform the action

  time.sleep(1)
  clear_output(wait=True)
  env.render()
    
  if done:
      break

env.close()